In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/allstate-purchase-prediction-challenge/train.csv.zip
/kaggle/input/allstate-purchase-prediction-challenge/sampleSubmission.csv
/kaggle/input/allstate-purchase-prediction-challenge/test_v2.csv.zip


In [14]:
# Allstate Purchase Prediction Challenge
# Author: Alessandro Mariani <alzmcr@yahoo.it>
# https://www.kaggle.com/c/allstate-purchase-prediction-challenge


import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import operator

from time import time
from sklearn import ensemble

import multiprocessing, operator
import pandas as pd, numpy as np

## Pickle FIX for multiprocessing using bound methods
## http://stackoverflow.com/questions/1816958/cant-pickle-type-instancemethod-when-using-pythons-multiprocessing-pool-ma/
from copy_reg import pickle
from types import MethodType
from time import time
from itertools import combinations
from sklearn import preprocessing

import scipy as sp, numpy as np, pandas as pd

# Cantor Pairing
def cantor(args):
    # Cantor Pairing - recursive call if more than 1 pair
    if len(args) > 2:
        x2 = cantor(args[1:])
        x1 = args[0]
    else:
        x1, x2 = args
    return int((0.5 * (x1 + x2)*(x1 + x2 + 1) + x2))

# Groups all columns of data into combinations of [degree]
def group_data(data, degree=3, hash=hash, NAMES=None): 
    init = time()
    new_data = []; combined_names = []
    m,n = data.shape
    for indicies in combinations(range(n), degree):
        new_data.append([hash(tuple(v)) for v in data[:,indicies]])
        if NAMES != None:
            combined_names.append( '+'.join([NAMES[indicies[i]] for i in range(degree)]) )
    print =("DONE! %.2fm",((time()-init)/60))
    if NAMES != None:
        return (np.array(new_data).T, combined_names)
    return np.array(new_data).T

# Return concatenated fields in a dataframe
# [1,2,3,4,5,6] => '123456'
def concat(df, columns):
    return np.array([''.join(x) for x in np.array(
        [np.array(df[col].values, dtype=str) for col in columns]).T])

# Breakfast Pirate Awesome State trick + some additions
def stateFix(encoders,df,c=['C','D','G'],verbose=False):
    # GA
    iGA = df.state == encoders['state'].transform(['GA'])[0]
    ifix = iGA&(df[c[0]]==1); df.ix[ifix,c[0]] = 2; nga1 = np.sum(ifix) #C
    ifix = iGA&(df[c[1]]==1); df.ix[ifix,c[1]] = 2; nga2 = np.sum(ifix) #D
    # FL
    iFL = df.state == encoders['state'].transform(['FL'])[0]
    ifix = iFL&(df[c[2]]<=2); df.ix[ifix,c[2]] = 3; nfl1 = np.sum(ifix) #G
    # OH
    iOH = df.state == encoders['state'].transform(['OH'])[0]
    ifix = iOH&(df[c[2]]==1); df.ix[ifix,c[2]] = 2; noh1 = np.sum(ifix) #G
    # ND
    iND = df.state == encoders['state'].transform(['ND'])[0]
    ifix = iND&(df[c[2]]!=2); df.ix[ifix,c[2]] = 2; nnd1 = np.sum(ifix) #G
    # SD
    iSD = df.state == encoders['state'].transform(['SD'])[0]
    ifix = iSD&(df[c[2]]!=2); df.ix[ifix,c[2]] = 2; nsd1 = np.sum(ifix) #G
    if verbose:
        print =("Fixed state law products. GA1:%i GA2:%i FL1:%i OH1:%i ND1:%i SD1:%i",(
            nga1, nga2, nfl1, noh1, nnd1, nsd1))

# Target variable expected value given a categorical feature
def expval(df,col,y,tfilter):
    tmp = pd.DataFrame(index=df.index)
    pb = df[tfilter][y].mean()                                              # train set mean
    tmp['cnt'] = df[col].map(df[tfilter][col].value_counts()).fillna(0)     # train set count
    tmp['csm'] = df[col].map(df[tfilter].groupby(col)[y].sum()).fillna(pb)  # train set sum
    tmp.ix[tfilter,'cnt'] -= 1                                              # reduce count for train set
    tmp.ix[tfilter,'csm'] -= df.ix[tfilter,y]                               # remove current value
    tmp['exp'] = ((tmp.csm+ pb*15) / (tmp.cnt+ 15)).fillna(pb)              # calculate mean including kn-extra 'average' samples 
    np.random.seed(1)
    tmp.ix[tfilter,'exp'] *= 1+.3*(np.random.rand(len(tmp[tfilter]))-.5) # add some random noise to the train set
    return tmp.exp

def prepare_data(shuffle=True):
    alltest = pd.read_csv('data\\test_v2.csv')
    test = alltest.set_index('customer_ID')
    alldata = pd.read_csv('data\\train.csv').set_index('customer_ID')

    # handy lists of features
    con = ['group_size','car_age','age_oldest','age_youngest','duration_previous','cost']
    cat = ['homeowner','car_value','risk_factor','married_couple','C_previous','state', 'location','shopping_pt']
    conf = ['A','B','C','D','E','F','G']; conf_f = [col+'_f' for col in conf]
    extra = []

    final_purchase = alldata[alldata.record_type == 1]          # final purchase
    data = alldata.join(final_purchase[conf], rsuffix='_f')     # creating training dataset with target features
    data = data[data.record_type == 0]                          # removing final purchase

    data['conf'] = concat(data,conf_f)                          # handy purchase plan 
    data['conf_init'] = concat(data,conf)                       # handy last quoted plan

    encoders = dict()
    data = data.append(test)

    # Fix NAs
    data['C_previous'].fillna(0, inplace=1)
    data['duration_previous'].fillna(0, inplace=1)
    data.location.fillna(-1, inplace=1);
    # Transform data to numerical data
    for col in ['car_value','risk_factor','state']:
        encoders[col] = preprocessing.LabelEncoder()
        data[col] = encoders[col].fit_transform(data[col].fillna(99))

    print =('Location substitution:'),
    ## get rid of very location, given the total count from train,cv and test set
    x = data[data.shopping_pt==2].location.value_counts()
    sub = data.location.map(x).fillna(0) < 5
    data.ix[sub,'location'] = data.state[sub]; print =('%.5f',sub.mean())

    # cost per car_age; cost per person; cost per state
    data['caCost'] = 1.*data.cost / (data.car_age+1)
    data['ppCost'] = 1.*data.cost / data.group_size
    data['stCost'] = data.state.map(data.groupby('state')['cost'].mean())
    extra.extend(['caCost','ppCost','stCost'])

    # average quote cost by G values
    data['costG'] = data['G'].map(data.groupby('G')['cost'].mean())
    extra.append('costG')

    # average quote cost by G & state values
    x = data.groupby(['G','state'])['cost'].mean()
    x = x.reset_index().set_index(['G','state']); x.columns = ['costStG']   # covert to DF
    data = data.merge(x,left_on=['G','state'],right_index=True,how='left')
    extra.append('costStG')

    # two way intersactino between state, G and shopping_pt
    print =("Grouping few 2-way interactions..."),
    grpTrn, c2 = group_data(data[['state','G','shopping_pt']].values,2,hash,['state','G','shopping_pt'])
    for i,col in enumerate(c2):
        encoders[col] = preprocessing.LabelEncoder()
        data[col] = encoders[col].fit_transform(grpTrn[:,i])
    extra.extend(c2)

    # expected value (arithmetic average) of G by state & location
    for col in ['state','location']:
        extra.append(col+'_exp')
        data[col+'_exp'] = expval(data,col,'G_f',-data.G_f.isnull())

    # previous G
    data['prev_G'] = data.G.shift(1); extra.append('prev_G')
    data.ix[data.shopping_pt == 1,'prev_G'] = data.ix[data.shopping_pt==1,'G']

    # separating training & test data
    test = data[data.conf.isnull()]; data = data[-data.conf.isnull()]

    # SHUFFLE THE DATASET, keeping the same customers transaction in order
    if shuffle:
        print =("Shuffling dataset..."),
        np.random.seed(9); ids = np.unique(data.index.values)
        rands = pd.Series(np.random.random_sample(len(ids)),index=ids)
        data['rand'] = data.reset_index()['customer_ID'].map(rands).values
        data.sort(['rand','shopping_pt'],inplace=1); print =("DONE!")

    # convert to int due to emtpy values in test set
    for col in conf_f: data[col] = np.array(data[col].values,dtype=np.int8)

    return data,test,con,cat,extra,conf,conf_f,encoders

        
def _pickle_method(method):
    func_name = method.im_func.__name__
    obj = method.im_self
    cls = method.im_class
    return _unpickle_method, (func_name, obj, cls)

def _unpickle_method(func_name, obj, cls):
    for cls in cls.mro():
        try:
            func = cls.__dict__[func_name]
        except KeyError:
            pass
        else:
            break
    return func.__get__(obj, cls)

class RandomForestsParallel(object):
    # class used to fit & predict in parallel minimizing memory usage
    rfs = []
    def __init__(self,N,ntree,maxfea,leafsize,N_proc=None):
        self.N = N
        self.ntree = ntree; self.maxfea = maxfea; self.leafsize = leafsize
        self.N_proc = N_proc if N_proc is not None else max(1,multiprocessing.cpu_count()-1)

        # fix pickling when using bound methods in classes
        pickle(MethodType, _pickle_method, _pickle_method)

    def _parallel_fit(self, rf):
        t = time()
        return rf.fit(self.X,self.y,self.w), (time()-t)/60.

    def _parallel_predict(self, rf):
        return rf.predict(self.X)
    
    def fit(self,X,y,w=None):
        # fit N random forest in parallel
        self.rfs = []; self.X = X; self.y = y
        self.w = np.ones(y.shape,dtype=bool) if w is None else w
        print =("fitting %i RFs using %i processes...",(self.N,self.N_proc)),

        args = [ensemble.RandomForestClassifier(
            n_estimators=self.ntree, max_features=self.maxfea,
            min_samples_leaf=self.leafsize,random_state=irf,
            compute_importances=1) for irf in range(self.N)]

        if self.N_proc > 1:
            pool = multiprocessing.Pool(self.N_proc)
            for i,(rf,irft) in enumerate(pool.imap(self._parallel_fit,args)):
                self.rfs.append(rf); print =("rf#%i %.2fm",(i,irft)),
            pool.terminate()
        else:
            for i,rf in enumerate(args):
                rf,irft = self._parallel_fit(rf)
                self.rfs.append(rf); print=("rf#%i %.2fm",(i,irft)),
                
        del self.X,self.y,self.w
        # set the importances of the features
        self.impf = self._calculate_impf(X.columns)

        return self
        
    def predict(self,X,single_process=True):
        # predict using all the random forest in self.rfs
        # single_process is set by default, as multiprocess predict is not
        # memory efficient and sometime time efficient (efficient smaller N)
        self.X = X
        if (not single_process) & (self.N_proc > 1):
            pool = multiprocessing.Pool(self.N_proc)
            allpreds = np.array([p for p in pool.imap(self._parallel_predict,self.rfs)]).T
            pool.terminate()
        else:
            allpreds = np.array([self._parallel_predict(rf) for rf in self.rfs]).T
            
        del self.X
        
        return allpreds

    def _calculate_impf(self, feature_names):
        # private method to calculate the average features importance
        return pd.Series(reduce(operator.add,[rf.feature_importances_ for rf in self.rfs]) / self.N, feature_names)

    def __repr__(self):
        return ("N:%i N_proc:%i ntree:%i maxfea:%i leafsize:%i fitted:%s",(    
            self.N, self.N_proc, self.ntree,self.maxfea,
            self.leafsize, 'Yes' if len(self.rfs) > 0 else 'No'))

from sklearn.model_selection import cross_validate,
from utils import prepare_data,
from parallel import RandomForestsParallel
from time import time

def majority_vote(baseline,model_predictions):
    # given a baseline and a matrix of prediction (#samples x #models)
    # if will return the prediction if 1+#models/2 agree on the same product
    # otherwise will return the baseline
    prcnt = np.vstack([np.bincount(p,minlength=5) for p in model_predictions])
    prmax = np.max(prcnt,axis=1) >= (1+(len(selected)/2))
    preds = baseline+0; preds[prmax] = np.argmax(prcnt[prmax],axis=1)
    return preds

def make_ptscores(y_true,y_pred,y_base,pt,vmask):
    # measure the increase of "plan" accuracy given a prediction for the product (G)
    return [np.mean(vmask[pt==ipt]&(y_true[pt==ipt] == y_pred[pt==ipt])) - np.mean(vmask[pt==ipt]&(y_true[pt==ipt] == y_base[pt==ipt])) for ipt in range(1,11)]

if __name__ == '__main__':
    ############################################################################
    ## SETTING #################################################################
    # submit: if 'True' create a submission file and train models for submission
    # N: number of models to build
    # NS: number of models to selected for majority vote
    # kfold: number of k-fold to perform, if not submitting
    # N_proc: number of process to spawn, default #CPU(s)-1
    # include_from_pt: minimum shopping_pt included in the data set
    # verbose_selection: print all details while selecting the model
    # tn: test set distrubution of shopping_pt (#10-11 merged)    
    ############################################################################
    submit = True; N = 50; NS = 9; kfold = 3; N_proc = None;
    include_from_pt = 1; verbose_selection = False
    tn = np.array([18943,13298,9251,6528,4203,2175,959,281,78])
    ############################################################################
    # Random Forest Setting ####################################################
    # Must be a list containg a tuple with (ntree,maxfea,leafsize)
    params = [(50,5,23)]
    # ex. [(x,5,23) for x in [35,50,75]] # [(50,x,23) for x in range(4,12)]
    # anything you'd like to try, here is the place for the modifications
    ############################################################################
    
    print=("Majority vote using %i models, selecting %i\n",(N,NS))
    # initialize data
    data,test,con,cat,extra,conf,conf_f,encoders = prepare_data()
    data = data[data.shopping_pt >=include_from_pt]; print=("Including from shopping_pt #%i\n",data.shopping_pt.min()),
    # features, target, weights (not used)
    X = data[con+cat+conf+extra]; y = data['G_f'] ; w = np.ones(y.shape)
    
    vmask = reduce(operator.and_,data[conf[:-1]].values.T==data[conf_f[:-1]].values.T)
    scores,imp,ptscores = {},{},{}
    for n,m,l in params:
        t = time();
        scores[(m,l)],imp[(m,l)],ptscores[(m,l)] = [],[],[]
        col_trscores,col_cvscores = [],[]

        # initialize the ensemble of forests to run in parallel
        # class is also structured to handle single-process 
        rfs = RandomForestsParallel(N, n, m, l, N_proc)
        
        # cross validation is use to find the best parameters
        for ifold,(itr,icv) in enumerate(cross_validation.KFold(len(y),kfold,indices=False)):
            if submit:
                # just a lame way to re-using the same code for fitting & selecting when submitting :)
                itr = np.ones(y.shape,dtype=bool); icv = -itr
                print=("\nHEY! CREATING SUBMISSION!\n")
            else:
                # redo expected value for the current training & cv set
                for c in [x for x in X.columns if x[-4:] == '_exp']:
                    X[c] = expval(data,c[:-4],'G_f',itr)           

            # fits the random forests at the same time
            rfs.fit(X[itr],y[itr],w[itr])

            print=("predicting..."),
            allpreds = rfs.predict(X)
            rftscores = []
            print=("selecting models...")
            for irf in range(len(rfs.rfs)):
                # SELECTION of the best random forest, even though probably
                # is just getting rid of very unlucky seeds ...
                pG = allpreds[:,irf]; ipt2 =  data.shopping_pt > 1
                ptscore = make_ptscores(y[icv],pG[icv],data.G[icv],data.shopping_pt[icv],vmask[icv])
                tptscore = make_ptscores(y[itr],pG[itr],data.G[itr],data.shopping_pt[itr],vmask[itr])
                rftscores.append((tn.dot(tptscore[1:]),irf))
                print =("%i,%i %.5f %.5f %.5f %.5f"),(
                    ifold,irf,
                    np.mean(pG[itr]==y[itr]),np.mean(vmask[itr]&(pG[itr]==y[itr])),
                    np.mean(pG[ipt2&itr]==y[ipt2&itr]),np.mean(vmask[ipt2&itr]&(pG[ipt2&itr]==y[ipt2&itr]))),
                if verbose_selection:
                    print =(" ".join(["%.5f" %pts for pts in ptscore])),
                    print =(" ".join(["%.5f" %pts for pts in tptscore])),
                print =("%.2f %.2f",(tn.dot(tptscore[1:]),tn.dot(ptscore[1:])))

            # select the best models for the majority vote
            rftscores.sort(reverse=1); selected = [x[1] for x in rftscores[:NS]]

            print =("counting votes...")
            # print also the score using all the models
            pG = majority_vote(data.G,allpreds)
            ptscore = make_ptscores(y[icv],pG[icv],data.G[icv],data.shopping_pt[icv],vmask[icv])
            # ifold,a : majority vote score using all models
            print =(str(ifold)+",a "+" ".join(["%.5f" %pts for pts in ptscore])+" %.2f" % tn.dot(ptscore[1:]))
            
            # results for selected models
            pG = majority_vote(data.G,allpreds[:,selected])
            ptscore = make_ptscores(y[icv],pG[icv],data.G[icv],data.shopping_pt[icv],vmask[icv])
            # ifold,s : majority vote score using selected models
            print =(str(ifold)+",s "+" ".join(["%.5f" %pts for pts in ptscore])+" %.2f" % tn.dot(ptscore[1:]))
            
            # append features importances & scores
            col_trscores.append(np.mean(pG[itr]==y[itr]))        # append train score
            col_cvscores.append(np.mean(pG[icv]==y[icv]))        # append cv score
            imp[(m,l)].append(rfs.impf)
            scores[(m,l)].append(tn.dot(ptscore[1:]))
            ptscores[(m,l)].append(ptscore)

            # skip any following fold if we're submitting
            if submit: break
        
        print=("%i %i %i\t %.2f %.2f %.4f %.4f %.2f - %.2fm",(
            n,m,l,
            np.mean(scores[(m,l)]), np.std(scores[(m,l)]),  # for best params & variance
            np.mean(col_trscores), np.mean(col_cvscores),   # use x diagnostic training set overfit
            tn.dot(np.mean(ptscores[(m,l)],axis=0)[1:])),    # score
            (time()-t)/60),                                 # k-fold time
        print =(" ".join(["%.5f" %pts for pts in np.mean(ptscores[(m,l)],axis=0)])),
        print =(" ".join(["%.5f" %pts for pts in np.std(ptscores[(m,l)],axis=0)]))
        
    if submit:
        # MAKE SUBMISSION
        # very complicated way to keep only the latest shopping_pt for each customer just to have everything in one row!!!!!11
        test = test[test.shopping_pt == test.reset_index().customer_ID.map(test.reset_index().groupby('customer_ID').shopping_pt.max())]
        Xt = test[con+cat+conf+extra]

        # TEST SET PREDICTION
        print =("now predicting on test set..."),
        allpreds = rfs.predict(Xt)
        test['pG'] = majority_vote(test.G,allpreds[:,selected]); print =("done")
        
        # Fix state law products, then concatenate to string
        stateFix(encoders,test,['C','D','pG'],1)
        test['plan'] = concat(test,['A','B','C','D','E','F','pG'])
        test['plan'].to_csv('submission\\majority_rfs%i_%i.%i_shuffle_GAfix_%iof%iof%i.csv' % (
            n,m,l,NS/2+1,NS,N),header=1)

        # features importances
        impf = rfs.impf; impf.sort()
        test.to_csv('submission.csv')

SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-14-1582cf12b45d>, line 263)